# vcfstats
# What does this do?

### Run rtg-tools vcfstats
- Get list of all variants.gvcf.gz files
- Get list of all rtg_vcfstats.txt files
- Convert each to lists of sample IDs
- Compare lists to find samples that are missing rtg_vcfstats.txt files
- Use make-batch-tsv-from-input-sample.py to generate dsub task file
- Run dsub to generate missing rtg_vcfstats.txt files

### Convert vcfstats result files to CSV format
- Get list of all rtg_vcfstats.txt files
- Get list of all rtg_vcfstats.txt.csv files
- 

### Concatenate CSV Files

# Code

## 0. Check that environment variables have been loaded correctly

Environment variables are imported from the mvp-profile.sh file. If this echo command does not return anything, try sourcing it from the command console. If any of the values are incorrect, change them in mvp-profile.sh, save it, and source it again.

In [1]:
#source /Users/pbilling/Documents/GitHub/test/mvp-on-gcp/mvp-profile.sh
echo "Date stamp: ${date_stamp}"
echo "Home directory: ${mvp_hub}"
echo "Project: ${mvp_project}"
echo "Bucket: ${mvp_bucket}"
echo "Zone: ${mvp_zone}"

Date stamp: 20170929
Home directory: /Users/pbilling/Documents/GitHub/test/mvp-on-gcp
Project: gbsc-gcp-project-mvp
Bucket: gbsc-gcp-project-mvp-phase-2-data
Zone: us-central1-*


## 1. Run rtg-tools vcfstats

#### Create file accounting directory it doesn't already exist

In [3]:
accounting_dir="${mvp_hub}/vcfstats/file-accounting/${date_stamp}"
mkdir -p ${accounting_dir}

dsub_inputs_dir="${mvp_hub}/vcfstats/dsub-inputs"

#### Get list of sample IDs for gvcf files that already exist on Google Cloud Storage

In [3]:
gsutil ls gs://${mvp_bucket}/data/bina-deliverables/*/*/VariantCalling/variants.gvcf.gz \
    > ${accounting_dir}/gs-gvcf-${date_stamp}.txt
cut -d'/' -f6 ${mvp_hub}/vcfstats/file-accounting/${date_stamp}/gs-gvcf-${date_stamp}.txt \
    > ${accounting_dir}/gs-gvcf-sample-ids-${date_stamp}.txt

#### Get list sample IDs for vcfstats files that already exist on Google Cloud Storage

In [4]:
gsutil ls gs://${mvp_bucket}/dsub/vcfstats/rtg-tools/objects/*_rtg_vcfstats.txt \
    > ${accounting_dir}/gs-vcfstats-rtg-${date_stamp}.txt
cut -d '/' -f8 ${accounting_dir}/gs-vcfstats-rtg-${date_stamp}.txt \
    | cut -d'_' -f1 > ${accounting_dir}/gs-vcfstats-rtg-sample-ids-${date_stamp}.txt

#### Get difference between lists of sample IDs to find out which samples are missing vcfstats files

In [5]:
diff --new-line-format="" --unchanged-line-format "" \
    <(sort ${accounting_dir}/gs-gvcf-sample-ids-${date_stamp}.txt) \
    <(sort ${accounting_dir}/gs-vcfstats-rtg-sample-ids-${date_stamp}.txt) \
    > ${accounting_dir}/gs-vcfstats-rtg-missing-sample-ids-${date_stamp}.txt
grep -F \
    -f ${accounting_dir}/gs-vcfstats-rtg-missing-sample-ids-${date_stamp}.txt \
    ${accounting_dir}/gs-gvcf-${date_stamp}.txt \
    > ${accounting_dir}/gs-vcfstats-rtg-missing-${date_stamp}.txt

: 1

#### Create dsub TSV task file to generate missing vcfstats files

In [6]:
${mvp_hub}/bin/make-batch-tsv-from-input-sample.py \
    -i ${accounting_dir}/gs-vcfstats-rtg-missing-${date_stamp}.txt \
    -t ${dsub_inputs_dir}/rtg-tools/gs-vcfstats-rtg-missing-${date_stamp}.tsv \
    -o gs://${mvp_bucket}/dsub/vcfstats/rtg-tools/objects \
    -s rtg_vcfstats.txt

Traceback (most recent call last):
  File "/Users/pbilling/Documents/GitHub/test/mvp-on-gcp/bin/make-batch-tsv-from-input-sample.py", line 88, in <module>
    main()
  File "/Users/pbilling/Documents/GitHub/test/mvp-on-gcp/bin/make-batch-tsv-from-input-sample.py", line 75, in main
    out_fh = open(table_file, 'w')
IOError: [Errno 2] No such file or directory: '/Users/pbilling/Documents/GitHub/test/mvp-on-gcp/vcfstats/dsub-inputs/rtg-tools/gs-vcfstats-rtg-missing-20170929.tsv'


: 1

#### Run RTG vcfstats dsub tasks

In [7]:
dsub \
    --zones "${mvp_zone}" \
    --project ${mvp_project} \
    --logging gs://${mvp_bucket}/dsub/vcfstats/rtg-tools/logs/${date_stamp} \
    --image gcr.io/${mvp_project}/rtg-tools:1.0 \
    --command 'rtg vcfstats ${INPUT} > ${OUTPUT}' \
    --tasks ${dsub_inputs_dir}/rtg-tools/gs-vcfstats-rtg-missing-${date_stamp}.tsv ${dsub_range} \
    --dry-run

bash: dsub: command not found


: 127

## 2. Convert vcfstats result files to CSV format

#### Get list of sample IDs for vcfstats files that already exist on Google Cloud Storage

In [10]:
gsutil ls gs://${mvp_bucket}/dsub/vcfstats/rtg-tools/objects/*_rtg_vcfstats.txt \
    > ${accounting_dir}/gs-vcfstats-rtg-${date_stamp}.txt
cut -d'/' -f8 ${accounting_dir}/gs-vcfstats-rtg-${date_stamp}.txt \
    | cut -d'_' -f1 > ${accounting_dir}/gs-vcfstats-rtg-sample-ids-${date_stamp}.txt

#### Get list of sample IDs for vcfstats CSV files that already exist on Google Cloud Storage

In [11]:
gsutil ls gs://${mvp_bucket}/dsub/vcfstats/text-to-table/objects/*_rtg_vcfstats.txt.csv \
    > ${accounting_dir}/gs-vcfstats-csv-${date_stamp}.txt
cut -d'/' -f8 ${accounting_dir}/gs-vcfstats-csv-${date_stamp}.txt \
    | cut -d'_' -f1 > ${accounting_dir}/gs-vcfstats-csv-sample-ids-${date_stamp}.txt

#### Get difference between lists of sample IDs to find out which samples are missing vcfstats files

In [13]:
diff \
    --new-line-format="" \
    --unchanged-line-format "" \
    <(sort ${accounting_dir}/gs-vcfstats-rtg-sample-ids-${date_stamp}.txt) \
    <(sort ${accounting_dir}/gs-vcfstats-csv-sample-ids-${date_stamp}.txt) \
    > ${accounting_dir}/gs-vcfstats-csv-sample-ids-missing-${date_stamp}.txt
grep -F -f \
    ${accounting_dir}/gs-vcfstats-csv-sample-ids-missing-${date_stamp}.txt \
    ${accounting_dir}/gs-vcfstats-rtg-${date_stamp}.txt \
    > ${accounting_dir}/gs-vcfstats-csv-missing-${date_stamp}.txt

: 1

#### Convert file list to dsub TSV files

In [14]:
${mvp_hub}/bin/make-batch-tsv-from-input-file.py \
    -i ${accounting_dir}/gs-vcfstats-csv-missing-${date_stamp}.txt \
    -t ${dsub_inputs_dir}/text-to-table/gs-vcfstats-csv-missing-${date_stamp}.tsv \
    -o gs://${mvp_bucket}/dsub/vcfstats/text-to-table/objects \
    -s csv \
    -c rtg_vcfstats \
    -e rtg-vcfstats-${date_stamp}

Traceback (most recent call last):
  File "/Users/pbilling/Documents/GitHub/test/mvp-on-gcp/bin/make-batch-tsv-from-input-file.py", line 77, in <module>
    main()
  File "/Users/pbilling/Documents/GitHub/test/mvp-on-gcp/bin/make-batch-tsv-from-input-file.py", line 62, in main
    out_fh = open(table_file, 'w')
IOError: [Errno 2] No such file or directory: '/Users/pbilling/Documents/GitHub/test/mvp-on-gcp/vcfstats/dsub-inputs/text-to-table/gs-vcfstats-csv-missing-20170929.tsv'


: 1

#### Run dsub task

In [15]:
dsub \
    --zones ${mvp_region} \
    --project ${mvp_project} \
    --logging gs://${mvp_bucket}/dsub/vcfstats/text-to-table/logs/${date_stamp} \
    --image gcr.io/${mvp_project}/text-to-table:0.2.0 \
    --command 'text2table -s ${SCHEMA} -o ${OUTPUT} -v series=${SERIES},sample=${SAMPLE_ID} ${INPUT}' \
    --tasks ${mvp_hub}/vcfstats/dsub-inputs/text-to-table/gs-vcfstats-csv-missing-${date_stamp}.tsv ${dsub_range} \
    --dry-run

bash: dsub: command not found


: 127

## 3. Concatenate CSV Files

#### Get new list of completed results files

In [16]:
gsutil ls gs://${mvp_bucket}/dsub/vcfstats/text-to-table/objects/*_rtg_vcfstats.txt.csv \
    > ${accounting_dir}/gs-vcfstats-csv-${date_stamp}.txt

#### Run dsub task

In [17]:
dsub \
    --zones ${mvp_zone} \
    --project ${mvp_project} \
    --logging gs://${mvp_bucket}/dsub/vcfstats/concat/logs/${date_stamp} \
    --image gcr.io/${mvp_project}/text-to-table:0.2.0 \
    --disk-size 100 \
    --script ${mvp_hub}/vcfstats/dsub-scripts/text-to-table.sh \
    --input INPUT_FILES=gs://${mvp_bucket}/dsub/vcfstats/text-to-table/objects/*_vcfstats.txt.csv \
    --output CONCAT_FILE=gs://${mvp_bucket}/dsub/vcfstats/concat/concat_vcfstats.txt.csv \
    --env SCHEMA=rtg_vcfstats \
    --env SERIES=vcfstats-${date_stamp} \
    --dry-run

bash: dsub: command not found


: 127